<a href="https://colab.research.google.com/github/sagunm/drug-discovery-project/blob/main/Data_collection_and_pre_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The ChEMBL Database is a database that contains curated bioactivity data of more than 2 million compounds. It is compiled from more than 76,000 documents, 1.2 million assays and the data spans 13,000 targets and 1,800 cells and 33,000 indications. [Data as of March 25, 2020; ChEMBL version 26].

In [2]:
#Install the ChEMBL web service package so that we can retrieve bioactivity data from the ChEMBL Database.
! pip install chembl_webresource_client

     |████████████████████████████████| 55 kB 2.2 MB/s 
     |████████████████████████████████| 596 kB 8.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Attempting uninstall: itsdangerous
    Found existing installation: itsdangerous 1.1.0
    Uninstalling itsdangerous-1.1.0:
      Successfully uninstalled itsdangerous-1.1.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
flask 1.1.4 requires itsdangerous<2.0,>=0.24, but you have itsdangerous 2.0.1 which is incompatible.


# **Importing all libraries**

In [3]:
import pandas as pd
from chembl_webresource_client.new_client import new_client

###**Search for target protein**

### Target search for coronavirus

In [4]:
#Target search for corona virus
target = new_client.target
target_query = target.search('coronavirus')
targets = pd.DataFrame.from_dict(target_query)
targets

cross_references  ...   tax_id
0                                                 []  ...    11119
1                                                 []  ...   227859
2                                                 []  ...    12663
3                                                 []  ...    11137
4  [{'xref_id': 'P0C6U8', 'xref_name': None, 'xre...  ...   227859
5                                                 []  ...  1335626
6  [{'xref_id': 'P0C6X7', 'xref_name': None, 'xre...  ...   227859
7                                                 []  ...  2697049

[8 rows x 9 columns]

### **Select and retrieve bioactivity data for *SARS coronavirus 3C-like proteinase* (fifth entry)**

In [5]:
#the selected target will then be the fifth entry
selected_target = targets.target_chembl_id[4]
selected_target

'CHEMBL3927'

Only bioactivity data for *coronavirus 3C-like proteinase* (CHEMBL3927) will be retrieved and reported as IC$_{50}$ values in nM (nanomolar) unit.

In [6]:
activity = new_client.activity
res= activity.filter(target_chembl_id=selected_target).filter(standard_type='IC50')

In [7]:
df = pd.DataFrame.from_dict(res)

In [8]:
df.head(3)

activity_comment  activity_id  ... upper_value value
0             None      1480935  ...        None   7.2
1             None      1480936  ...        None   9.4
2             None      1481061  ...        None  13.5

[3 rows x 45 columns]

In [9]:
len(df.index)

133

In [10]:
df.standard_type.unique()

array(['IC50'], dtype=object)

In [11]:
#saving the resulting bioactivity data to a CSV file
df.to_csv('bioactivity_data.csv', index=False)

### **Copying files need to Google Drive**

In [12]:
# mounting Drive into Colab
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [13]:
#creating a folder for data
! mkdir "/content/gdrive/My Drive/Colab Notebooks/data"

In [14]:
! cp bioactivity_data.csv "/content/gdrive/My Drive/Colab Notebooks/data"

In [15]:
! ls -l "/content/gdrive/My Drive/Colab Notebooks/data"

total 69
-rw------- 1 root root 70334 Nov 27 23:36 bioactivity_data.csv


In [16]:
! ls

bioactivity_data.csv  drive  gdrive  sample_data


In [17]:
! head bioactivity_data.csv

activity_comment,activity_id,activity_properties,assay_chembl_id,assay_description,assay_type,assay_variant_accession,assay_variant_mutation,bao_endpoint,bao_format,bao_label,canonical_smiles,data_validity_comment,data_validity_description,document_chembl_id,document_journal,document_year,ligand_efficiency,molecule_chembl_id,molecule_pref_name,parent_molecule_chembl_id,pchembl_value,potential_duplicate,qudt_units,record_id,relation,src_id,standard_flag,standard_relation,standard_text_value,standard_type,standard_units,standard_upper_value,standard_value,target_chembl_id,target_organism,target_pref_name,target_tax_id,text_value,toid,type,units,uo_units,upper_value,value
,1480935,[],CHEMBL829584,In vitro inhibitory concentration against SARS coronavirus main protease (SARS CoV 3C-like protease),B,,,BAO_0000190,BAO_0000357,single protein format,Cc1noc(C)c1CN1C(=O)C(=O)c2cc(C#N)ccc21,,,CHEMBL1139624,Bioorg. Med. Chem. Lett.,2005,"{'bei': '18.28', 'le': '0.33', 'lle': '3.25', 'sei': '5.90'}

### **Handling missing data**

In [18]:
#Dropping compounds that have missing standard_value column values
df2 = df[df.standard_value.notna()]
df2

activity_comment  activity_id  ... upper_value  value
0               None      1480935  ...        None    7.2
1               None      1480936  ...        None    9.4
2               None      1481061  ...        None   13.5
3               None      1481065  ...        None  13.11
4               None      1481066  ...        None    2.0
..               ...          ...  ...         ...    ...
128             None     12041507  ...        None   10.6
129             None     12041508  ...        None   10.1
130             None     12041509  ...        None   11.5
131             None     12041510  ...        None   10.7
132             None     12041511  ...        None   78.9

[133 rows x 45 columns]

### **Data pre-processing of the bioactivity data**

#### **Classifying compounds as active, inactive or intermediate**

The IC50 unit represents the bioactivity data. Compounds having values of less than 1000 nM will be considered to be **active** while those greater than 10,000 nM will be considered to be **inactive**. As for those values in between 1,000 and 10,000 nM will be referred to as **intermediate**. 

In [19]:
bioactivity_class = []
for i in df2.standard_value:
  if float(i) >= 10000:
    bioactivity_class.append('inactive')
  elif float(i) <= 1000:
    bioactivity_class.append('active')
  else:
    bioactivity_class.append('intermediate')

In [20]:
# Making a list with all the molecule_chembl_id
mol_cid = []
for i in df2.molecule_chembl_id:
  mol_cid.append(i)

In [21]:
# Makng a list with all the canonical_smiles
canonical_smiles = []
for i in df2.canonical_smiles:
  canonical_smiles.append(i)

In [22]:
# Makng a list with all the standard_value
standard_value = []
for i in df2.standard_value:
  standard_value.append(i)

In [23]:
# Combining the lists bioactivity_class, mol_cid, canonical_smiles, and standard_value into a dataframe
data_t = list(zip(mol_cid, canonical_smiles, bioactivity_class, standard_value))
df3 = pd.DataFrame(data_t, columns=["molecule_chembl_id", "canonical_smiles", "bioactivity_class", "standard_value"])
df3

molecule_chembl_id  ... standard_value
0         CHEMBL187579  ...         7200.0
1         CHEMBL188487  ...         9400.0
2         CHEMBL185698  ...        13500.0
3         CHEMBL426082  ...        13110.0
4         CHEMBL187717  ...         2000.0
..                 ...  ...            ...
128      CHEMBL2146517  ...        10600.0
129       CHEMBL187460  ...        10100.0
130       CHEMBL363535  ...        11500.0
131       CHEMBL227075  ...        10700.0
132        CHEMBL45830  ...        78900.0

[133 rows x 4 columns]

In [24]:
#Or 
selected = ['molecule_chembl_id', 'canonical_smiles', "standard_value"]
df4 = df2[selected]
df4

molecule_chembl_id  ... standard_value
0         CHEMBL187579  ...         7200.0
1         CHEMBL188487  ...         9400.0
2         CHEMBL185698  ...        13500.0
3         CHEMBL426082  ...        13110.0
4         CHEMBL187717  ...         2000.0
..                 ...  ...            ...
128      CHEMBL2146517  ...        10600.0
129       CHEMBL187460  ...        10100.0
130       CHEMBL363535  ...        11500.0
131       CHEMBL227075  ...        10700.0
132        CHEMBL45830  ...        78900.0

[133 rows x 3 columns]

In [25]:
pd.concat([df4, pd.Series(bioactivity_class).rename('bioactivity_class')], axis=1)

molecule_chembl_id  ... bioactivity_class
0         CHEMBL187579  ...      intermediate
1         CHEMBL188487  ...      intermediate
2         CHEMBL185698  ...          inactive
3         CHEMBL426082  ...          inactive
4         CHEMBL187717  ...      intermediate
..                 ...  ...               ...
128      CHEMBL2146517  ...          inactive
129       CHEMBL187460  ...          inactive
130       CHEMBL363535  ...          inactive
131       CHEMBL227075  ...          inactive
132        CHEMBL45830  ...          inactive

[133 rows x 4 columns]

In [26]:
# Saving dataframe to csv
df4.to_csv('bioactivity_preprocessed_data.csv', index=False)

In [27]:
! ls -l

ls: cannot access 'drive': Transport endpoint is not connected
total 92
-rw-r--r-- 1 root root 70334 Nov 27 23:35 bioactivity_data.csv
-rw-r--r-- 1 root root  9337 Nov 28 00:13 bioactivity_preprocessed_data.csv
d????????? ? ?    ?        ?            ? drive
drwx------ 5 root root  4096 Nov 27 23:35 gdrive
drwxr-xr-x 1 root root  4096 Nov 18 14:36 sample_data


In [28]:
! cp bioactivity_preprocessed_data.csv "/content/gdrive/My Drive/Colab Notebooks/data"

In [29]:
! ls "/content/gdrive/My Drive/Colab Notebooks/data"

bioactivity_data.csv  bioactivity_preprocessed_data.csv
